In [1]:
import pandas as pd 
import numpy as np 

In [2]:
path = '/home/hoan/Python/Code/AloNhaDat.csv'
df = pd.read_csv(path)
df.sample(3)

,city,date,direction,district,floors,front_road,id,length,price,property_legal_document,rooms,size,type_adv,type_real_estate,ward,width
78439,Hồ Chí Minh,06/11/2021,_,Quận 3,3,12m,9300749,20m,36 tỷ,Sổ hồng/ Sổ đỏ,6,160 m,Cần bán,Nhà mặt tiền,Phường 4 ...,8m
107700,Hồ Chí Minh,07/11/2021,Bắc,Quận 7,3,12m,9270452,29m,"43,3 tỷ",Sổ hồng/ Sổ đỏ,10,493 m,Cần bán,"Biệt thự, nhà liền kề",Phường Bình Thuận ...,17m
41613,Hồ Chí Minh,02/11/2021,_,Quận 9,---,6m,9252197,32m,"1,35 tỷ",Sổ hồng/ Sổ đỏ,---,162 m,Cần bán,"Đất thổ cư, đất ở",Phường Long Thạnh Mỹ ...,5m


In [3]:
df.columns = [column.split(' ')[0] for column in df.columns]

In [4]:
df_01 = df.copy()

In [5]:
def rm_price_unit(value):

    temp = ''
    mill_unit = {3: '.', 2: '.0', 1: '.00'}
    thous_unit = {3: '.000', 2: '.0000', 1: '.00000'}
    
    value = value.replace(',', '.')
    value = value.replace(' ', '')

    if 'tỷ' in value:
        ls_val = value.split('tỷ')
        temp = temp + ls_val[0]
        if 'triệu' in value:
            ls_val = ls_val[1].split('triệu')
            if 'ngàn' in value:
                ls_val = ls_val[1].split('ngàn')
                temp = temp + ls_val[0]
    elif 'triệu' in value:
            ls_val = value.split('triệu')
            if '.' in value:
                return float(ls_val[0])
            temp = temp + mill_unit[len(ls_val[0])] + ls_val[0]
            if 'ngàn' in value:
                ls_val = ls_val[1].split('ngàn')
                temp = temp + ls_val[0]
    elif 'ngàn' in value:
        ls_val = value.split('ngàn')
        temp = temp + thous_unit[len(ls_val[0])] + ls_val[0]
    else:
        #print(value)
        return np.nan

    temp = float(temp)
    if 'm' in value:
        temp *=-1
        
    return temp

In [6]:
ls_error = {}
def rm_measure_unit(value, column):
    if column == 'width': 
        error = {'1O': 10,4.5: 4.5,'4.5l': 4.5,'=': np.nan,'.7.1': 7.1,'tuycan': np.nan,3.75: 3.75,'4.3.': 4.3,
                'a': np.nan,5.15: 5.15,'4.2ình': 10.7,'4.2(nhà.': 4.2,'4.5x': 4.5,5.25: 5.25,'4.7=6': 5,'3.2\\6.8': 5,
                '.3.5': 3.5,'.6.6': 6.6,'4.5.': 4.5,'4.6.': 4.6,'.3.6': 3.6,'4.1+7.3': 5.7,6.0: 6,'4.9.8': 4.98,'.': np.nan,
                '3.5.': 3.5,6.5: 6.5,'4:2': 4.2,'5.5.': 5.5,4.25: 4.25,'Ã': np.nan,'4l.2': 4.2,6.8: 6.8}

    elif column == 'length':
        error = {'65-75': 70,'75;65': 70,'7p': 7,'5.×29': 5.29,'tu`ycan': np.nan,
                '1u': np.nan,'3u': np.nan,'`26': 26,'.6.3': 6.3,'`7': 7,'7/5': 7.5,
                '10/63': 10.63,'``15': 15,'2-0': 2,'1p': np.nan,'10\\': 10,'<25': 20,
                '2”': 5.08,'89?': 89,'.': np.nan,'`19': 19,'15/7': 15.7,'2r': np.nan,
                'G?n': np.nan,'13.5.': 13.5,'14.5.': 14.5,'19.5.': 19.5,'5.6.': 5.6,
                '16.5.': 16.5,'17.5.': 17.5,'10p': 10, 70.0: 70,1.0: 0, 'tuycan':np.nan}

    elif column == 'front_road':
        error = {'QL': 26, '>10': 10, '6-8': 7, '5q': 5, 'Xe': np.nan, "5'": 5, '`13': 13,'2-': 2,
                '8i': 8,'3f': 3,'`12': 12,'I': np.nan,'2-3': 2.5,'3 5': 4,'`10': 10,'6l5': 6.5,
                '16M': 16,'1"': 2.54,'6i': 6,'3.1/5': 3.2,'xe': np.nan,'12a': 12,'...': np.nan,
                '.': np.nan,'>4': 5,'ô': np.nan,'>5': 6,'Nhà': np.nan,'Qp': np.nan,'Ai': np.nan,
                7.0: 7, 2.5: 2.5, '..': np.nan, '+': np.nan,'gay': np.nan,'Xi1': np.nan,'12`': 12,
                '>3': 4,'1w': np.nan,'aâ': np.nan,'---        ': np.nan, '-': np.nan, 'H?': np.nan,
                 '5-6': 5.5, '5..': 5, 5.5: 5.5}
    elif column == 'size':
        error = {'1.000.000': 1000000,'3.500.000': 3500000,'3.400.000': 3400000,
        '20.000.000': 20000000,'12.681.000': 12681000,'KXĐ': np.nan,'10.000.000': 10000000,
        '3.060.000': 3060000,'3.118.880': 3118880,'1.141.920': 1141920,'4.280.000': 4280000,
        '4.850.000': 4850000,'100.000.000': 100000000,'120.000.000': 120000000,
        '5.800.000': 5800000,'6.888.888': 6888888}
    else:
        raise Exception('Do not support column: {}'.format(column))

    if '---' in value:
        return np.nan
    try:
        value = value.replace(' ', '').replace(' ', '').replace("'","").replace('`', '')
        value = value.replace(',', '.').replace('..', '.')
        value = value.split('m')[0]
        
        if '-' in value:
            ls_val = value.split('-')
            value = ( float(ls_val[0]) + float(ls_val[1]) ) / 2
        if '/' in value:
            ls_val = value.split('/')
            value = ( float(ls_val[0]) + float(ls_val[1]) ) / 2
        value = float(value)
    except:
        # if value not in error:
        #     ls_error[value] = value
        value = error[value]
        value = float(value)
    return value

In [7]:
df_01['price'] = df_01['price'].apply(rm_price_unit)

In [8]:
df_01['width'] = df_01['width'].apply(lambda x:rm_measure_unit(value=x,column='width'))

In [9]:
df_01['length'] = df_01['length'].apply(lambda x:rm_measure_unit(value=x,column='length'))

In [10]:
df_01['front_road'] = df_01['front_road'].apply(lambda x:rm_measure_unit(value=x,column='front_road'))

In [11]:
df_01['size'] = df_01['size'].apply(lambda x:rm_measure_unit(value=x,column='size'))

In [12]:
df_01['floors'] = df_01['floors'].apply(lambda x: np.nan if '---' in x else float(x))

In [13]:
df_01['rooms'] = df_01['rooms'].apply(lambda x: np.nan if '---' in x else float(x))

In [14]:
df_01['direction'] = df_01['direction'].apply(lambda x: np.nan if '_' in x else x)

In [15]:
df_01['property_legal_document'] = df_01['property_legal_document'].apply(lambda x: np.nan if '---' in x else x)

In [16]:
df_01.drop(labels=['id', 'type_adv'], axis=1, inplace=True)

In [17]:
df_01.sample(10)

,city,date,direction,district,floors,front_road,length,price,property_legal_document,rooms,size,type_real_estate,ward,width
46355,Hồ Chí Minh,03/11/2021,NaN,Quận Tân Bình,2.0,8.0,25.0,10.00,Sổ hồng/ Sổ đỏ,4.0,100.0,Nhà trong hẻm,Phường 14 ...,4.0
23562,Hồ Chí Minh,23/10/2021,NaN,Quận 12,6.0,NaN,NaN,2.04,NaN,2.0,72.0,Căn hộ chung cư,Phường Tân Thới Nhất ...,NaN
52735,Hồ Chí Minh,04/11/2021,NaN,Quận Bình Tân,3.0,10.0,18.0,4.80,Sổ hồng/ Sổ đỏ,4.0,72.0,Nhà trong hẻm,Phường Bình Hưng Hòa A ...,4.0
86074,Hồ Chí Minh,06/11/2021,NaN,Quận 1,NaN,20.0,NaN,14.00,Sổ hồng/ Sổ đỏ,NaN,84.0,Nhà mặt tiền,Phường Tân Định ...,NaN
49617,Hồ Chí Minh,03/11/2021,NaN,Quận 8,1.0,5.0,13.0,4.23,Sổ hồng/ Sổ đỏ,2.0,47.0,Nhà trong hẻm,Phường 3 ...,3.6
125250,Hồ Chí Minh,06/11/2021,NaN,Quận 5,3.0,20.0,18.0,30.00,NaN,6.0,72.0,Nhà mặt tiền,Phường 15 ...,4.0
32895,Hồ Chí Minh,01/11/2021,NaN,Quận 10,5.0,30.0,14.0,40.00,Sổ hồng/ Sổ đỏ,7.0,84.0,Nhà mặt tiền,Phường 12 ...,6.0
122429,Hồ Chí Minh,08/11/2021,NaN,Quận Gò Vấp,3.0,20.0,24.0,11.50,Sổ hồng/ Sổ đỏ,4.0,101.0,Nhà mặt tiền,Phường 5 ...,4.2
82879,Hồ Chí Minh,06/11/2021,Tây,Quận 7,1.0,25.0,28.0,18.50,Sổ hồng/ Sổ đỏ,4.0,117.0,Nhà mặt tiền,Phường Bình Thuận ...,4.2
32401,Hồ Chí Minh,31/10/2021,NaN,Quận Bình Thạnh,4.0,6.0,18.8,13.80,NaN,5.0,85.0,Nhà trong hẻm,Phường 5 ...,4.5


In [25]:
df_02 = df_01.copy()

In [95]:
for column in ['city', 'district', 'ward', 'property_legal_document', 'type_real_estate']:
    print(column)
    print(df_02[column].value_counts())
    print('----------------------------------------')
    

city
Hồ Chí Minh               111462
Đồng Nai                    9816
Bình Dương                  4217
P6 Quận 3 Hồ Chí Minh          1
Name: city, dtype: int64
----------------------------------------
district
Quận Gò Vấp                    13827
Quận Tân Bình                  12455
Quận Bình Thạnh                 9384
Quận Tân Phú                    9285
Quận 10                         6870
Quận 1                          6863
Quận Phú Nhuận                  6424
Quận Bình Tân                   5751
Quận 3                          5514
Thành phố Biên Hoà              4832
Quận 7                          4065
Thành phố Thủ Đức               3901
Quận 9                          3570
Quận 2                          3483
Quận 12                         2958
Huyện Củ Chi                    2857
Huyện Bình Chánh                2508
Quận 11                         2101
Quận 5                          2038
Huyện Nhà Bè                    1998
Quận 8                          1840
Quận 6     